<p><img src="https://files.cdn.thinkific.com/file_uploads/133292/images/a9e/c1a/bc3/P02.jpg"></p>

`MCSDSC02P01V3______`

`MCSDSC02P01V2______`

`MCSDSELIMC02P01V1010621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Отлично! Проект принят с первого раза!

## Постановка задачи

Необходимо изучить данные сервиса с тестового запуска на территории дизайн-завода Флакон и Хлебозавода №9 и ответить на вопросы, поставленные менеджером проекта:

* На какие кафе приходится больше всего заказов? А на какие дни?
* Чаще ли приложением пользуются именно в обеденное время?
* Насколько большими компаниями  (или наоборот поодиночке) приходят в кафе?

## Импорт данных и библиотек

In [5]:
import pandas as pd

Прочитаем файл `bez_ocheredi.csv` и сохраним его в переменную `bo_data`.

In [6]:
bo_data = pd.read_csv('../../data/bez_ocheredi.csv')

Получим первые 10 строк таблицы.

In [7]:
bo_data.head(11)

,userID,ресторан,weekday,ordered,Столик для
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3
5,51e0962387f7,Там Ем,Friday,12:30,2
6,c4f260a2b48a,NaN,Saturday,15:00,6
7,6b71309d6a8a,Крафтейнер,Monday,10:30,1
8,c5b969cb63a2,Бар Сибирь,Wednesday,15:30,2
9,b1570962e68c,Бар Сибирь,Wednesday,12:30,3


Получим общую информацию о данных из таблицы.

In [8]:
bo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userID        1024 non-null   object
 1    ресторан     959 non-null    object
 2    weekday      1024 non-null   object
 3    ordered      1024 non-null   object
 4    Столик для   1024 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 40.1+ KB


### Выводы

Столбцы в `bo_data` содержат следующую информацию:

* userID - ID пользователя, который заказывал столик
* ресторан - наименование ресторана
* weekday - день, на который был заказ столик
* ordered -  время, на которое был заказн столик
* Столик для - количество человек 

* Всего таблица из файла содержит 1024 строки и 5 столбцов
* Все столбцы кроме "Столик для" имеют данные типа object, а столбец "Столик для" имеет тип integer
* В колонке "ресторан" есть 959 non-null значений и 65 null значений

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Тут все ОК!

## Предобработка данных

Получим названия столбцов.

In [9]:
bo_data.columns

Index(['userID ', ' ресторан ', ' weekday', ' ordered ', ' Столик для '], dtype='object')

Обращаем внимание, что помимо названий на разном языке, в них есть пробелы, которые затрудняют доступ к данным. Переименуем столбцы и проверим результат.

In [10]:
bo_data.set_axis(['userID', 'restaurant', 'weekday', 'order_time', 'people_quantity'], axis='columns', inplace=True)
bo_data.columns

Index(['userID', 'restaurant', 'weekday', 'order_time', 'people_quantity'], dtype='object')

Еще раз проверим данные на наличие пропусков, просуммировав их.

In [11]:
bo_data.isnull().sum()

userID              0
restaurant         65
weekday             0
order_time          0
people_quantity     0
dtype: int64

Менеджер проекта предупреждал, что некоторые заказы заводились на тестовую карточку кафе без названия, поэтому есть смысл заполнить такой пропуск словом admin.

In [12]:
bo_data['restaurant'] = bo_data['restaurant'].fillna('admin')

Проверяем, что пропусков не осталось.

In [13]:
bo_data.isnull().sum()

userID             0
restaurant         0
weekday            0
order_time         0
people_quantity    0
dtype: int64

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [14]:
bo_data.duplicated().sum()

24

In [15]:
bo_data = bo_data.drop_duplicates().reset_index(drop=True)

In [16]:
bo_data.duplicated().sum()

0

In [17]:
bo_data

,userID,restaurant,weekday,order_time,people_quantity
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3
...,...,...,...,...,...
995,4b136de2c16a,Iskra Bar,Saturday,11:30,2
996,9e46b91cc547,Бар Сибирь,Thursday,10:00,2
997,5ca79042c44d,Там Ем,Thursday,14:00,3
998,e0f9dfd1726c,Там Ем,Tuesday,17:30,1


Скорее всего, такие полные дубликаты говорят о сбое при выгрузке данных. Необходимо сообщить об этом.

Еще одна проблема, о которой предупреждал менеджер проекта — дубликаты карточки Iskra Bar с неправильными названием бара - "Искра" и "Искр". На них заводились реальные заказы, поэтому необходимо вычислить их и переименовать на правильное название — Iskra Bar.

Объявим функцию `replace_restaurant()`, которая заменяет неправильное название, а потом проверяет успешность выполнения замены подсчетом (если замена прошла успешно — результат подсчета должен быть 0).

In [18]:
def replace_restaurant(dataset, bar_name, new_bar_name):
    dataset.replace(bar_name, new_bar_name, inplace = True)
    return dataset[dataset['restaurant'] == bar_name].count()

Передаем функции названия 'Искра' и 'Искр'.

In [19]:
replace_restaurant(bo_data, 'Искра', 'Iskra Bar')

userID             0
restaurant         0
weekday            0
order_time         0
people_quantity    0
dtype: int64

In [20]:
replace_restaurant(bo_data, 'Искр', 'Iskra Bar')

userID             0
restaurant         0
weekday            0
order_time         0
people_quantity    0
dtype: int64

### Выводы

* были исправлены наименования столбцов и удалены ненужные символы в наименованиях столбцов
* в столбце "restaurant" были выявлены 65 значений без наименования, которые были заменены на слово "admin"
* были удалены дубликаты вместе со строками и был сделан их пересчет и строк получилось 1000, а не 1024
* были исправилены наименования Iskra Bar и приведены к одному типу

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Также без замечаний!

## Распределение заказов в течение недели

Считаем, на какие кафе приходится больше всего заказов.

In [21]:
bo_data['restaurant'].value_counts()

Iskra Bar     287
Там Ем        200
Бар Сибирь    159
Северия       107
Крафтейнер     78
Коворкафе      66
Азия           62
admin          41
Name: restaurant, dtype: int64

Группируем по `weekday` и считаем, на какие дни приходится больше всего заказов.

In [22]:
bo_data.groupby('weekday')['restaurant'].count().sort_values(ascending=False)

weekday
Tuesday      251
Wednesday    242
Thursday     180
Monday       141
Friday       110
Saturday      52
Sunday        24
Name: restaurant, dtype: int64

Объявляем функцию, которая возвращает количество заказов для ресторана в определенный день.

In [23]:
def numbers_of_orders_per_day(dataset, restaurant, day):
    numbers_of_orders = dataset[(dataset['restaurant'] == restaurant) & (dataset['weekday'] == day)]
    return numbers_of_orders['restaurant'].count()

Передадим ей заведения Iskra Bar и Там Ем — два флагманских места, по которым просили результаты прежде всего — и понедельник, среду и пятницу в качестве дня недели.

In [24]:
numbers_of_orders_per_day(bo_data, 'Iskra Bar', 'Monday')

48

In [25]:
numbers_of_orders_per_day(bo_data, 'Iskra Bar', 'Wednesday')

53

In [26]:
numbers_of_orders_per_day(bo_data, 'Iskra Bar', 'Friday')

29

In [27]:
numbers_of_orders_per_day(bo_data, 'Там Ем', 'Monday')

25

In [28]:
numbers_of_orders_per_day(bo_data, 'Там Ем', 'Wednesday')

52

In [29]:
numbers_of_orders_per_day(bo_data, 'Там Ем', 'Friday')

24

Сведем результат в таблицу.

In [30]:
isk_mon = numbers_of_orders_per_day(bo_data, 'Iskra Bar', 'Monday')
isk_wed = numbers_of_orders_per_day(bo_data, 'Iskra Bar', 'Wednesday')
isk_fri = numbers_of_orders_per_day(bo_data, 'Iskra Bar', 'Friday')
tam_mon = numbers_of_orders_per_day(bo_data, 'Там Ем', 'Monday')
tam_wed = numbers_of_orders_per_day(bo_data, 'Там Ем', 'Wednesday')
tam_fri = numbers_of_orders_per_day(bo_data, 'Там Ем', 'Friday')

data = [
    ['Iskra Bar',isk_mon, isk_wed, isk_fri],
    ['Там Ем', tam_mon, tam_wed, tam_fri]
]
columns = ['Ресторан', 'Понедельник', 'Среда', 'Пятница']

result = pd.DataFrame(data=data, columns=columns)
result

,Ресторан,Понедельник,Среда,Пятница
0,Iskra Bar,48,53,29
1,Там Ем,25,52,24


### Выводы

* больше всего заказов приходится на кафе Iskra bar
* самые частые дени, на которые производятся заказы, вторник и среда
* Iskra Bar - самое посещаемое заведение в понедельник, среду и пятницу

## Распределение заказов в течение дня

Сравним количества заказов, сделанных в обеденное и вечернее время — с 12 до 16 и с 16 до 22. Для этого объявим функцию, которая принимает данные и 2 аргумента времени.

In [31]:
lunch_start = '12:00'
lunch_end = '15:59'
dinner_start = '16:00'
dinner_end = '21:59'

def lunch_and_dinner_orders(dataset, start_time, end_time):
    time = (dataset['order_time'] >= start_time) & (dataset['order_time'] < end_time)
    
    orders = dataset[time]
    return orders['order_time'].count() 

Передадим промежуток с 12:00 и 16:00

In [32]:
lunch_and_dinner_orders(bo_data, lunch_start, lunch_end)

543

И с 16:00 до 22:00

In [33]:
lunch_and_dinner_orders(bo_data, dinner_start, dinner_end)

338

### Выводы

* в обеденное время столиков заказывается больше, чем в вечернее
* вероятно, что столики в обед заказывают больше офисные работники

## Типичное количество человек за столиком

Выясним, для какого количества человек обычно заказывают столик в разное время на примере Iskra Bar и Там Ем. Для этого напишем функцию, которая принимает данные, название ресторана, 2 времени и считает моду.

In [34]:
def number_of_persons(dataset, rest_name, start_time, end_time):
    restaurant = dataset['restaurant'] == rest_name
    time = (dataset['order_time'] >= start_time) & (dataset['order_time'] < end_time)
    median_num = dataset[restaurant & time]
    
    return median_num['people_quantity'].mode()

Проверим функцию на двух ресторанах, а также дневном и вечернем промежутке времени.

In [35]:
number_of_persons(bo_data, 'Iskra Bar', lunch_start, lunch_end)

0    2
Name: people_quantity, dtype: int64

In [36]:
number_of_persons(bo_data, 'Iskra Bar', dinner_start, dinner_end)

0    1
Name: people_quantity, dtype: int64

In [37]:
number_of_persons(bo_data, 'Там Ем', lunch_start, lunch_end)

0    2
Name: people_quantity, dtype: int64

In [38]:
number_of_persons(bo_data, 'Там Ем', dinner_start, dinner_end)

0    1
Name: people_quantity, dtype: int64

### Выводы

В обеденное время в двух самых популярных заведениях столик бронируется на 2-х человек в чаще всего, а в вечернее время на одного. В результате этого можно предположить, что в заведениях редко бронируются столики для больших компаний и вечеринок.

## Общие выводы

1. Была проведена работа по корректировке таблицы: заменены пустые ячейки, удалены дубликаты, рассчитаны значения  по дням, по количеству заказов в течение дня и количество человек
2. На основании полученных данных можно сказать:
    * самые популярные для заказа заведения: Iskra Bar - 287 и Там Ем - 200 заказов в неделю
    * самые часто посещаемы дни - Вторник - 251, Среда - 242, Четверг - 180
    * больше всего столики бронируются в обеденное время - 543, чем в вечернее - 338
    * чаще всего обедают люди по 2 человека, а ужинают по одному. На основании этого можно сказать, что приложение не очень удобно для организации больших вечеринок и похода компании в заведения. Зато очень удобно забронировать столик на одного или двух человек, чтобы быстро пообедать или поужинать и пойти дальше по своим делам.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Отличная работа!